In [304]:
import glob
import os
import pandas as pd
from pathlib import Path
import dateutil.parser as dparser

In [305]:
def parse_header(lines):
    header = {}

    for line in lines:
        if not line.isspace():
            if ":" in line:
                new_key, new_val = [x.strip() for x in line.split(':',maxsplit=1)]
                try:
                    new_val = dparser.parse(new_val, fuzzy=True)
                except:
                    print("foo")

                header[new_key] = new_val
    
    return header

In [306]:
def parse_data(lines):
    # UGLY, could be one-liner, or?
    head = lines[0].split()
    params_no = len(head)

    data = dict.fromkeys(head)

    # UGLY, I believe there shoould be a way of doing it without for loop
    for key in data:
        data[key] = []

    for line in lines[1:]:
        if not "Bkgnd" in line:
            if len(line.split()) == params_no:
                # UGLY, do it in one line
                for i in range(params_no):
                    data[head[i]].append(line.split()[i])

    return data

In [307]:
def parse_RPT(rpt_file):
    my_file = open(rpt_file, 'r')
    lines = my_file.readlines()

    lookup = "Bkgnd"
    for line in lines:
        #print("line is")
        #print(line)
        if lookup in line:
            split_num = lines.index(line)
            print("line to break in")
            print(line)
            break

    header_lines = lines[0:split_num]
    data_lines = lines[split_num:-1]

    header = parse_header(header_lines)
    data = parse_data(data_lines)
    
    return header, data

In [308]:
if not os.path.isdir("raw_reports"):
    raise Exception("Folder 'raw_reports' not found. Consider creating it and moving RPT files there.")

In [309]:
Path("./parsed_reports").mkdir(parents=True, exist_ok=True)

In [310]:
for rpt_file in glob.iglob("./raw_reports/*.RPT"):
    print(rpt_file)
    header, data = parse_RPT(rpt_file)

    meta_df = pd.DataFrame(header, index=[0])
    datetime_columns = []
    meta_df[datetime_columns] = pd.to_datetime(meta_df[datetime_columns])
    rpt_df = pd.DataFrame(data)
    name = (rpt_file.split('.')[-2]).split('/')[-1]
    meta_df.to_csv(f"parsed_reports/meta_{name}.csv")
    rpt_df.to_csv(f"parsed_reports/{name}.csv")

./raw_reports/M5-C1-Udepl-3.RPT
line to break in
   Pk IT  Energy      Area   Bkgnd  FWHM Channel  Left  PW  Cts/Sec  %err   Fit

foo
foo
foo
foo
foo
foo
foo
foo
foo
foo
foo
foo
foo
foo
foo
foo


ValueError: to assemble mappings requires at least that [year, month, day] be specified: [day,month,year] is missing

In [245]:
meta_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 0 to 0
Data columns (total 29 columns):
 #   Column                                                 Non-Null Count  Dtype 
---  ------                                                 --------------  ----- 
 0   Filename                                               1 non-null      object
 1   Report Generated On                                    1 non-null      object
 2   Sample Title                                           1 non-null      object
 3   Sample Description                                     1 non-null      object
 4   Sample Identification                                  1 non-null      object
 5   Sample Type                                            1 non-null      object
 6   Sample Geometry                                        1 non-null      object
 7   Peak Locate Threshold                                  1 non-null      object
 8   Peak Locate Range (in channels)                        1 non-nul

In [241]:
rpt_file

'./raw_reports/1M-28-30.RPT'

In [246]:
meta_df["Report Generated On"]

0    24.11.2020 10
Name: Report Generated On, dtype: object

In [285]:
line = "   ofslkslk   :            nvkslknsvlksvnjlkjn              "
new_key, new_val = [x.strip() for x in line.split(':',maxsplit=1)]

In [287]:
new_val

'nvkslknsvlksvnjlkjn'

In [278]:
y = [x.strip() for x in ["  aa  ", " bb         "]]
print(y)

['aa', 'bb']
